In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

In [2]:
regex_name = r'(([A-Z])\w+)'

In [21]:
# CHANGE THIS FOR EACH PROCESSING

DF_IN = lambda i: 'experiments-final{}/'.format(i)
DF_OUT = 'experiments-average/'
RUNS = 9

In [4]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

In [5]:
_approach = SOUTHBOUND

_approach_terms = [
    'Left',
    'Through',
    'Right'
]

_cell_path = [
    (CELL_SOURCE,0,_approach),
    (CELL_NORMAL,0,_approach),
    (CELL_NORMAL,1,_approach),
    (CELL_NORMAL,2,_approach),
    (CELL_MOVEMENT,THROUGH_TURN,_approach),
    S_mapping((CELL_MOVEMENT,THROUGH_TURN,_approach))[0]
]

_movement_labels = {
    (2,LEFT_TURN,NORTHBOUND): 'NBL',
    (2,RIGHT_TURN,NORTHBOUND): 'NBR',
    (2,THROUGH_TURN,NORTHBOUND): 'NBT',
    (2,LEFT_TURN,SOUTHBOUND): 'SBL',
    (2,RIGHT_TURN,SOUTHBOUND): 'SBR',
    (2,THROUGH_TURN,SOUTHBOUND): 'SBT',
    (2,LEFT_TURN,EASTBOUND): 'EBL',
    (2,RIGHT_TURN,EASTBOUND): 'EBR',
    (2,THROUGH_TURN,EASTBOUND): 'EBT',
    (2,LEFT_TURN,WESTBOUND): 'WBL',
    (2,RIGHT_TURN,WESTBOUND): 'WBR',
    (2,THROUGH_TURN,WESTBOUND): 'WBT',
}

_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

_col_rename = {
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value',
    'Average time': 'travel_time',
    'Average speed': 'average_speed',
}

ROUTE_MAPPING = {
    'approach0_lane1-North_to_East-exit1_lane1': ('North', 'Left'),
    'approach0_lane2-North_to_South_Lane2_Lane3-exit2_lane2': ('North', 'Through'),
    'approach0_lane3-North_to_West-exit3_lane3': ('North', 'Right'),
    'approach1_lane1-East_to_South-exit2_lane1': ('East', 'Left'),
    'approach1_lane2-East_to_West_Lane2_Lane3-exit3_lane2': ('East', 'Through'),
    'approach1_lane3-East_to_North-exit0_lane3': ('East', 'Right'),
    'approach2_lane1-South_to_West-exit3_lane1': ('South', 'Left'),
    'approach2_lane2-South_to_North_Lane2_Lane3-exit0_lane2': ('South', 'Through'),
    'approach2_lane3-South_to_East-exit1_lane3': ('South', 'Right'),
    'approach3_lane1-West_to_North-exit0_lane1': ('West', 'Left'),
    'approach3_lane2-West_to_East_Lane2_Lane3-exit1_lane2': ('West', 'Through'),
    'approach3_lane3-West_to_South-exit2_lane3': ('West', 'Right')
}

def movement_paths(approach):
    return [
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,LEFT_TURN,approach),
            S_mapping((CELL_MOVEMENT,LEFT_TURN,approach))[0]
        ],
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,THROUGH_TURN,approach),
            S_mapping((CELL_MOVEMENT,THROUGH_TURN,approach))[0]
        ],
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,RIGHT_TURN,approach),
            S_mapping((CELL_MOVEMENT,RIGHT_TURN,approach))[0]
        ]
    ]

In [6]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

# Dataframe Averaging

## Loading Dataframes

In [7]:
DF_PATH = DF_IN(5)

In [8]:
# Results of the MILP solver only (Expected)

df_milp = pd.read_pickle(DF_PATH + 'milp-realtime-results.pkl')

df_milp.head()

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,165.902929,3069.088608,51.278481,709.539027,450,True,1.00,0.00,0.00,Delay priority,0
5,540.855196,3069.088608,51.278481,-666.881330,450,True,0.50,0.00,0.50,Delay-Flow priority,0
3,475.487772,3069.088608,51.278481,258.865157,450,True,0.50,0.50,0.00,Delay-Throughput priority,0
6,752.900086,3069.088608,51.278481,-503.438553,450,True,0.33,0.33,0.33,Equal priority,0
2,573.475265,3265.265823,52.278481,-2055.801688,450,False,0.00,0.00,1.00,Flow priority,0


In [9]:
# Results of the LocalSim simulator only (Actual, Aggregate)
# Delay, Throughput

df_ave_delay = pd.read_pickle(DF_PATH + 'ave_delay.pkl')
df_total_delay = pd.read_pickle(DF_PATH + 'total_delay.pkl')
df_total_thru = pd.read_pickle(DF_PATH + 'total_throughput.pkl')

df_total_thru.head()

level_1,"(1, 0, 0)","(1, 0, 1)","(1, 0, 2)","(1, 0, 3)",demand,new_model,alpha,beta,gamma,model_type
0,964.0,1007.0,1045.0,1076.0,450,True,1.0,0.0,0.0,Delay priority
1,977.0,927.0,1059.0,1068.0,450,True,0.0,1.0,0.0,Throughput priority
2,1013.0,962.0,1005.0,1118.0,450,True,0.0,0.0,1.0,Flow priority
3,1056.0,1002.0,932.0,1134.0,450,True,0.5,0.5,0.0,Delay-Throughput priority
4,1074.0,1077.0,962.0,1123.0,450,True,0.0,0.5,0.5,Throughput-Flow priority


In [10]:
# Results of the LocalSim simulator only (Actual, Aggregate)
# Speed, Travel Time

df_ave_speed = pd.read_pickle(DF_PATH + 'ave_speed.pkl')
df_ave_travel_time = pd.read_pickle(DF_PATH + 'ave_travel_time.pkl')

df_ave_speed.head()

,average_speed,approach,movement,demand,new_model,alpha,beta,gamma,model_type
0,18.52,North,Left,450,True,1.0,0.0,0.0,Delay priority
1,35.90,North,Through,450,True,1.0,0.0,0.0,Delay priority
2,76.59,North,Right,450,True,1.0,0.0,0.0,Delay priority
3,13.83,East,Left,450,True,1.0,0.0,0.0,Delay priority
4,22.28,East,Through,450,True,1.0,0.0,0.0,Delay priority


In [11]:
# Comparing computed and measured values (Expected vs Actual)

df_delay_comp = pd.read_pickle(DF_PATH + 'delay_comparison.pkl')
df_thru_comp = pd.read_pickle(DF_PATH + 'throughput_comparison.pkl')

df_delay_comp.head()

,demand,model_type,expected,actual,epoch
95,450,Delay priority,3069.088608,619.710887,0
94,450,Delay priority,7380.430380,5409.827475,1
93,450,Delay priority,9401.316456,11593.014466,2
92,450,Throughput priority,3500.000000,492.159880,0
91,450,Throughput priority,9466.481013,7037.276468,1


In [33]:
# Casewise Cumulative Delay, Throughput, and CVCC (Actual, Per Case)

df_cumu_delay_tmp = pd.read_pickle(DF_PATH + 'cumu-delay/delay_d450_a0_b0.5_c0.5.pkl')
df_cumu_thru_tmp = pd.read_pickle(DF_PATH + 'cumu-thru/thru_d450_a0_b0.5_c0.5.pkl')
df_cvcc_tmp = pd.read_pickle(DF_PATH + 'cvcc/cvcc_d450_a0_b0.5_c0.5.pkl')

df_cumu_thru_tmp.head()

level_1,"(1, 0, 0)","(1, 0, 1)","(1, 0, 2)","(1, 0, 3)"
t,,,,
60,19.0,18.0,16.0,13.0
120,64.0,63.0,55.0,53.0
180,137.0,127.0,117.0,116.0
240,233.0,219.0,207.0,209.0
300,360.0,341.0,326.0,333.0


## 1. Average route information (Speed, Travel Time)

In [26]:
_df_cols = ['approach', 'movement', 'demand', 'new_model', 'alpha', 'beta', 'gamma', 'model_type']

_df_ave_speeds = []
_df_ave_traveltimes = []

for i in range(1,RUNS+1):
    _df_ave_speeds.append(pd.read_pickle(DF_IN(i) + 'ave_speed.pkl'))
    _df_ave_traveltimes.append(pd.read_pickle(DF_IN(i) + 'ave_travel_time.pkl'))
    
_df_tmp1 = pd.concat(_df_ave_speeds)
_df_tmp2 = pd.concat(_df_ave_traveltimes)

df_agg_speeds = _df_tmp1.groupby(_df_cols).mean().reset_index()
df_agg_traveltimes = _df_tmp2.groupby(_df_cols).mean().reset_index()

In [27]:
df_agg_speeds.to_pickle(DF_OUT + 'agg_speeds.pkl')
df_agg_traveltimes.to_pickle(DF_OUT + 'agg_traveltimes.pkl')

## 2. Average survey zone values (Delay, Throughput)

In [25]:
_df_cols = ['demand', 'new_model', 'alpha', 'beta', 'gamma', 'model_type']

_df_ave_delays = []
_df_total_delays = []
_df_total_thrus = []

for i in range(1,RUNS+1):
    _df_ave_delays.append(pd.read_pickle(DF_IN(i) + 'ave_delay.pkl'))
    _df_total_delays.append(pd.read_pickle(DF_IN(i) + 'total_delay.pkl'))
    _df_total_thrus.append(pd.read_pickle(DF_IN(i) + 'total_throughput.pkl'))
    
_df_tmp1 = pd.concat(_df_ave_delays)
_df_tmp2 = pd.concat(_df_total_delays)
_df_tmp3 = pd.concat(_df_total_thrus)

df_agg_ave_delays = _df_tmp1.groupby(_df_cols).mean().reset_index()
df_agg_total_delays = _df_tmp2.groupby(_df_cols).mean().reset_index()
df_agg_total_thrus = _df_tmp3.groupby(_df_cols).mean().reset_index()

In [28]:
df_agg_ave_delays.to_pickle(DF_OUT + 'agg_ave_delays.pkl')
df_agg_total_delays.to_pickle(DF_OUT + 'agg_total_delays.pkl')
df_agg_total_thrus.to_pickle(DF_OUT + 'agg_total_thrus.pkl')

## 3. Average cumulative delay and throughput

In [34]:
_df_cols = ['demand', 'new_model', 'alpha', 'beta', 'gamma', 'model_type']

for demand in _demands:
    for weights in _weights:
        _df_cumu_delays = []
        _df_cumu_thrus = []

        for i in range(1,RUNS+1):
            _df_cumu_delays.append(pd.read_pickle(DF_IN(i) + 'cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights)))
            _df_cumu_thrus.append(pd.read_pickle(DF_IN(i) + 'cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights)))

        _df_tmp1 = pd.concat(_df_cumu_delays)
        _df_tmp2 = pd.concat(_df_cumu_thrus)

        df_agg_cumu_delays = _df_tmp1.groupby(_df_tmp1.index).mean()
        df_agg_cumu_thrus = _df_tmp2.groupby(_df_tmp2.index).mean()
        
        df_agg_cumu_delays.to_pickle(DF_OUT + 'cumu-delays/delays_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_agg_cumu_thrus.to_pickle(DF_OUT + 'cumu-thrus/thrus_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))

## 4. Average MILP results

In [39]:
_df_cols = ['demand', 'new_model', 'alpha', 'beta', 'gamma', 'model_type', 'epoch']

_df_ave_milp_results = []

for i in range(1,RUNS+1):
    _df_ave_milp_results.append(pd.read_pickle(DF_IN(i) + 'milp-realtime-results.pkl'))
    
_df_tmp1 = pd.concat(_df_ave_milp_results)

df_ave_milp_results = _df_tmp1.groupby(_df_cols).mean().reset_index()

In [40]:
df_ave_milp_results.to_pickle(DF_OUT + 'ave_milp_results.pkl')